## ЗАДАЧА. 
### Построить прогнозную модель для оценки уровня продаж предприятия оптовой торговли.

__Входными__ данными для модели являются показатели с характеристиками покупателей и исторические сведения об продажах за период с 2005 - 2016 гг.

__Модель__ должна корректно (с приемлемой точностью) предсказывать уровень продаж по покупателям на определенный день.

__Гипотеза__ - рабочая гипотеза состоит в том, что признаками, влияющими на объем продаж являются характеристики (свойства) покупателя, история покупок покупателя и характеристики периодов покупок.

### Этапы решения задачи (рис.)

## Раздел I. Сбор и поготовка данных

__В задачи этого этапа входит извлечение исторических данных по продажам товаров в различных филиалах компании, слияние этих данных, контроль и первичная очистка.__

В наличии имеются 2 файла, хранящие информацию накладных в формате dbf.

Требуется конвертировать эти файлы в формат датафреймов, сформировать таблицу и устранить дублирование.

### TICKETS  - файл накладных
- T_PP.DBF - накладные по филиалу №1 (280М)
- T_PN.DBF - накладные пр филиалу №2 (165М)

набор данных содержит информацию о продажах сигарет за период 2008-2016гг. Каждая запись - это информация из накладной, которая выписывается покупателю. Одна строка содержит сведения о продаже одного товара, т.е. если в накладной несколько товаров,то будет нескольео строк с одинаковым номером накладной.

на выходе необходимо получить датафрейм, в котром будут следующие поля:
#### Описание полей:
- __CUST_NAM__ - имя покупателя
- __CUST_NO_L6__ - код покупателя
- __CUST_CAT__ - категория покупателя
- __SA_HDR_TIC__ - номер накладной
- __SA_HDR_DAT__ - дата накладной
- __SA_HDR_AMT__ - сумма накладной ($)
- __ITEM_NO__ - код товара
- __ITEM_DESC__ - наименование товара
- __SA_LIN_QTY__ - количество товара
- __SA_LIN_PRC__ - цена за ед. 
- __SA_LIN_COS__ - стоимость за ед.


In [1]:
import pandas as pd
from simpledbf import Dbf5      # библиотека для работы с dbf-форматом

### a) извлечение и консолидация данных

In [ ]:
# загрузка данных филиала №1
dbf = Dbf5('T_PP.DBF')
data_PP = Dbf5.to_dataframe(dbf)

# загрузка данных филиала №2
dbf = Dbf5('T_PN.DBF')
data_PN = Dbf5.to_dataframe(dbf)

In [ ]:
# слияние импортированных датафреймов
tickets = pd.concat([data_PN, data_PP], ignore_index=True)
print 'минимальная дата =',tickets.SA_HDR_DAT.min(); 
print 'максимальная дата =',tickets.SA_HDR_DAT.max()

In [ ]:
# удалить дублированные строки
tickets = tickets.drop_duplicates()

In [ ]:
tickets.shape     # размер датафрейма (строк, столбцов)

In [ ]:
tickets.head(3)   # вывод первых 3-х строк

In [ ]:
tickets.tail(3)   # вывод последних 3-х строк

In [ ]:
tickets.info()    # вывод общей информации о датафрейме

In [ ]:
# выгрузить датафрейм на диск в виде csv-файла 
tickets.to_csv('tickets.csv', index=False)

### б) агрегация данных (создание рабочего датафрейма)

In [ ]:
# загрузить `tickets`
tickets = pd.read_csv('tickets.csv', low_memory=False)

In [ ]:
# выбираем необходимые колонки (строим 'макет' рабочего датафрейма)
order_columns = ['CUST_NAM','CUST_NO_L6', 'SA_HDR_TIC', 'SA_HDR_DAT', 'SA_HDR_AMT', 
                 'CUST_CAT', 'CUST_PROFI', 'CUST_PRO01',
                 'CUST_PRO02', 'CUST_PRO03', 'CUST_PRO04', 'CUST_PRO05']

orders_tmp = tickets[order_columns]

In [ ]:
orders_tmp.shape

In [ ]:
# в временном файле уберем дублированные записи
orders = orders_tmp.drop_duplicates()

In [ ]:
orders.shape    # 'свернули' датафрейм почти в 10 раз

In [ ]:
orders.shape[0]    # количество накладных (продаж) для анализа

In [ ]:
orders.info()

In [ ]:
# сохраним 'orders' на диск для следующего этапа
orders.to_csv('orders_1.csv', index=False)